In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("spotify_millsongdata.csv")

In [5]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [7]:
df.shape

(57650, 4)

In [8]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [9]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [10]:
df.head(10)

,artist,song,text
0,Cat Stevens,Pop Star,Yes I'm going to be a pop star. \r\nYes I'm g...
1,Depeche Mode,A Pain That I'm Used To,I'm not sure what I'm looking for anymore \r\...
2,Who,Time Is Passing,I'm playing my guitar while my sister bangs th...
3,Stevie Wonder,Land Of La La,You were brought up in a small town \r\nWhere...
4,Tina Turner,Poor Fool,I want to tell all of you \r\nThat ain't doin...
5,Within Temptation,Hand Of Sorrow,The child without a name grew up to be the han...
6,Diana Ross,A Hard Day's Night,It's been a hard day's night \r\nAnd I've bee...
7,Hanson,Let You Go,You were amazing in ways I never knew \r\nIt'...
8,Zebrahead,Here's To You,Here she comes to ruin my day again \r\nRight...
9,Weird Al Yankovic,Germs,Sometimes I really want to be alone \r\nBut t...


In [11]:
df['text'][0]

"Yes I'm going to be a pop star.  \r\nYes I'm going to be a pop star, now.  \r\nYes I'm going to be a pop star.  \r\nOh mama, mama see me, mama, mama see me  \r\nI'm a pop star.  \r\n  \r\nYes, I'm going on the T.V. now.  \r\nYes, I'm going on the T.V. now.  \r\nYes, I'm going on the T.V. now.  \r\nOh mama, mama see me, mama, mama see me  \r\non the T.V.  \r\n  \r\nYes, I'm going on my first gig.  \r\nYes, I'm going on my first gig.  \r\nYes, I'm going on my first gig.  \r\nOh mama, mama see me, mama, mama see me  \r\non my first gig.  \r\n  \r\nNow listen to me,  \r\nLa da na la, na da la  \r\n  \r\nWell, I'm going to the cold bank, cold bank.  \r\nYes, I'm going to the cold bank.  \r\nYes, I'm going to the cold bank.  \r\nOh mama, mama see me, mama, mama see me  \r\nat the cold bank.  \r\n  \r\nMama see me!  \r\nLa da na la, na da la  \r\n  \r\nWell, I'm coming, coming, coming home now.  \r\nYes, I'm coming, coming, coming home now.  \r\nYes, I'm coming, coming, coming home now.  \r\

In [24]:
df = df.sample(5000)

In [13]:
df.shape

(5000, 3)

In [14]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [15]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [16]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [19]:
similarity[0]

array([1.        , 0.00478706, 0.00627524, ..., 0.        , 0.02219209,
       0.08722235])

In [20]:
df[df['song'] == 'Crying Over You']

,artist,song,text
649,ABBA,Crying Over You,i 'm waitin ' for you babi i 'm sit all alon i...


In [21]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [22]:
recommendation('Crying Over You')

["Cryin' Drunk",
 'Am I Blue',
 "Blues For X'mas",
 'Morning Dew',
 'THE BLUES COME AROUND',
 "The End Of The Lyin'",
 'Come In From The Outside',
 'Midnight Blue',
 'Blue Christmas',
 'You Move Me',
 'Living Doll',
 'Anything For You',
 "Lord I'm Gonna Love You",
 'Blue Christmas',
 "Baby's Callin' Me Home",
 'Strength Of My Life',
 'Baby Blue',
 'Drunken Wisdom',
 'Use Somebody',
 'Almost Blue']

In [23]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))